# ENGR-E 533: Deep Learning Systems
## Homework 2

### Khandokar Md. Nayem (knayem@iu.edu)
### Apr 3, 2018

### Import necessary files and set environment parameters
My assigned Node is `r-005` and GPU `0`.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.utils.np_utils import to_categorical

import tensorflow as tf
import numpy as np
import librosa
import glob

Using TensorFlow backend.


### Global Variables

Directory names, file formates.

In [2]:
PATH_directory = '/N/u/knayem/data/timit-homework/'

PATH_train = 'tr/'
PATH_val = 'v/'
PATH_test = 'te/'
PATH_denoise = 'te/'

CLEAN_format_train = 'trs*.wav'
NOISE_format_train = 'trn*.wav'
MIX_format_train = 'trx*.wav'

CLEAN_format_val = 'vs*.wav'
NOISE_format_val = 'vn*.wav'
MIX_format_val = 'vx*.wav'

MIX_format_test = 'tex*.wav'

Max_RNN = 5 # Number of Time Stamps in a RNN 

### File Names

Since there are huge number of files for both training, validation and testing, for convinience we write them in files. These are the name of the file.

In [3]:
TRAIN_S = 'TRAIN_S.txt'
TRAIN_N = 'TRAIN_N.txt'
TRAIN_X_cmplx = 'TRAIN_X_cmplx.txt'
TRAIN_X = 'TRAIN_X.txt'

VAL_S = 'VAL_S.txt'
VAL_S_cmplx = 'VAL_S_cmplx.txt'
VAL_N = 'VAL_N.txt'
VAL_X_cmplx = 'VAL_X_cmplx.txt'
VAL_X = 'VAL_X.txt'


TEST_S = 'TEST_S.txt'
TEST_N = 'TEST_N.txt'
TEST_X_cmplx = 'TEST_X_cmplx.txt'
TEST_X = 'TEST_X.txt'


### Load a .wav file
A function to get the Complex and Magnitude Spectum of a file. 

In [4]:
def preprossed_data(file_name):
    
    sn, sr=librosa.load(file_name, sr=None)
    X=librosa.stft(sn, n_fft=1024, hop_length=512)
    X_mag = np.abs(X)
#     print('X_mag',X_mag.shape)
    return X, X_mag

### Save Train Data  

Here we open the files, and read the Train Dataset. We need to run this portion for the first time only. This file reading needs lots fo time. After saving in file once, we can load data from files unless you want complex version of Noisy Speech. Complex numbers are hard to retrive from files.

In [ ]:
DATA_train_s = []
DATA_train_n = []
DATA_train_x = []
DATA_train_x_cmplx = []

with open(TRAIN_S,'wb') as fs, open(TRAIN_N,'wb') as fn, open(TRAIN_X_cmplx,'wb') as fx_cmplx, open(TRAIN_X,'wb') as fx: 
    
    for file_s, file_n, file_x in zip(sorted(glob.glob(PATH_directory+PATH_train+CLEAN_format_train)),sorted(glob.glob(PATH_directory+PATH_train+NOISE_format_train)),sorted(glob.glob(PATH_directory+PATH_train+MIX_format_train))):
        _,s = preprossed_data(file_s)
        DATA_train_s.append(np.array(s))
        np.savetxt(fs, s, fmt='%.5f')
        fs.write(b'\n')
        
        _,n = preprossed_data(file_n)
        DATA_train_n.append(np.array(n))
        np.savetxt(fn, n, fmt='%.5f')
        fn.write(b'\n')
        
        x_cmplx,x = preprossed_data(file_x)
        DATA_train_x.append(np.array(x))
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        DATA_train_x_cmplx.append(np.array(x_cmplx))
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
        
fs.close()
fn.close()
fx_cmplx.close()
fx.close()

### Save Validate Data

Here we open the files, and read the Validation Dataset. To evaluate SNR of validation data, we need to run this.

In [5]:
DATA_val_s = []
DATA_val_s_cmplx = []
DATA_val_n = []
DATA_val_x_cmplx = []
DATA_val_x = []

with open(VAL_S,'wb') as fs, open(VAL_N,'wb') as fn, open(VAL_X_cmplx,'wb') as fx_cmplx, open(VAL_X,'wb') as fx, open(VAL_S_cmplx,'wb') as fs_cmplx: 
    count = 1
    for file_s, file_n, file_x in zip(sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)),sorted(glob.glob(PATH_directory+PATH_val+NOISE_format_val)),sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
        s_cmplx,s = preprossed_data(file_s)
        DATA_val_s.append(np.array(s))
        np.savetxt(fs, s, fmt='%.5f')
        fs.write(b'\n')
        
        DATA_val_s_cmplx.append(np.array(s_cmplx))
        np.savetxt(fs_cmplx, s_cmplx, fmt='%.5f')
        fs_cmplx.write(b'\n')
        
        _,n = preprossed_data(file_n)
        DATA_val_n.append(np.array(n))
        np.savetxt(fn, n, fmt='%.5f')
        fn.write(b'\n')
        
        x_cmplx,x = preprossed_data(file_x)
        DATA_val_x.append(np.array(x))
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        DATA_val_x_cmplx.append(np.array(x_cmplx))
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
        print(count, '...')
        count += 1
        
fs.close()
fs_cmplx.close()
fn.close()
fx_cmplx.close()
fx.close()

1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...
10 ...
11 ...
12 ...
13 ...
14 ...
15 ...
16 ...
17 ...
18 ...
19 ...
20 ...
21 ...
22 ...
23 ...
24 ...
25 ...
26 ...
27 ...
28 ...
29 ...
30 ...
31 ...
32 ...
33 ...
34 ...
35 ...
36 ...
37 ...
38 ...
39 ...
40 ...
41 ...
42 ...
43 ...
44 ...
45 ...
46 ...
47 ...
48 ...
49 ...
50 ...
51 ...
52 ...
53 ...
54 ...
55 ...
56 ...
57 ...
58 ...
59 ...
60 ...
61 ...
62 ...
63 ...
64 ...
65 ...
66 ...
67 ...
68 ...
69 ...
70 ...
71 ...
72 ...
73 ...
74 ...
75 ...
76 ...
77 ...
78 ...
79 ...
80 ...
81 ...
82 ...
83 ...
84 ...
85 ...
86 ...
87 ...
88 ...
89 ...
90 ...
91 ...
92 ...
93 ...
94 ...
95 ...
96 ...
97 ...
98 ...
99 ...
100 ...
101 ...
102 ...
103 ...
104 ...
105 ...
106 ...
107 ...
108 ...
109 ...
110 ...
111 ...
112 ...
113 ...
114 ...
115 ...
116 ...
117 ...
118 ...
119 ...
120 ...
121 ...
122 ...
123 ...
124 ...
125 ...
126 ...
127 ...
128 ...
129 ...
130 ...
131 ...
132 ...
133 ...
134 ...
135 ...
136 ...
137 ...
138 ...
139 

1035 ...
1036 ...
1037 ...
1038 ...
1039 ...
1040 ...
1041 ...
1042 ...
1043 ...
1044 ...
1045 ...
1046 ...
1047 ...
1048 ...
1049 ...
1050 ...
1051 ...
1052 ...
1053 ...
1054 ...
1055 ...
1056 ...
1057 ...
1058 ...
1059 ...
1060 ...
1061 ...
1062 ...
1063 ...
1064 ...
1065 ...
1066 ...
1067 ...
1068 ...
1069 ...
1070 ...
1071 ...
1072 ...
1073 ...
1074 ...
1075 ...
1076 ...
1077 ...
1078 ...
1079 ...
1080 ...
1081 ...
1082 ...
1083 ...
1084 ...
1085 ...
1086 ...
1087 ...
1088 ...
1089 ...
1090 ...
1091 ...
1092 ...
1093 ...
1094 ...
1095 ...
1096 ...
1097 ...
1098 ...
1099 ...
1100 ...
1101 ...
1102 ...
1103 ...
1104 ...
1105 ...
1106 ...
1107 ...
1108 ...
1109 ...
1110 ...
1111 ...
1112 ...
1113 ...
1114 ...
1115 ...
1116 ...
1117 ...
1118 ...
1119 ...
1120 ...
1121 ...
1122 ...
1123 ...
1124 ...
1125 ...
1126 ...
1127 ...
1128 ...
1129 ...
1130 ...
1131 ...
1132 ...
1133 ...
1134 ...
1135 ...
1136 ...
1137 ...
1138 ...
1139 ...
1140 ...
1141 ...
1142 ...
1143 ...
1144 ...
1145 ...
1

### Save Test Data

Here we open the files, and read the Test Dataset. To construct Test output files, we need to run this.

In [ ]:
DATA_test_x_cmplx = []
DATA_test_x = []

with open(TEST_S,'wb') as fs, open(TEST_N,'wb') as fn, open(TEST_X_cmplx,'wb') as fx_cmplx, open(TEST_X,'wb') as fx: 
    count = 1
    for file_x in sorted(glob.glob(PATH_directory+PATH_test+MIX_format_test)):
        x_cmplx,x = preprossed_data(file_x)
        DATA_test_x.append(np.array(x))
        np.savetxt(fx, x, fmt='%.5f')
        fx.write(b'\n')
        
        DATA_test_x_cmplx.append(np.array(x_cmplx))
        np.savetxt(fx_cmplx, x_cmplx, fmt='%.5f')
        fx_cmplx.write(b'\n')
        
        print(count,'...')
        count += 1
        
fx_cmplx.close()
fx.close()

### Load Data

Function to load data from files.

In [6]:
def load_file(file_name):
    
    with open(file_name) as f:
        lines=f.readlines()
        print(len(lines))
        sentence_full=[]
        count = 0
        sentence=[]
        for line in lines:

            if count < 513:
                if count ==0:
                    sentence=np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    count+=1
                else:
                    myarray = np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    sentence=np.concatenate((sentence, myarray), axis=0)
                    count+=1
            else:
                sentence_full.append(sentence) 
                count=0
                sentence=[]
                
        return sentence_full

Load the Train files.

In [7]:
DATA_train_s = load_file(TRAIN_S)
DATA_train_n = load_file(TRAIN_N)
DATA_train_x_cmplx = load_file(TRAIN_X_cmplx)
DATA_train_x = load_file(TRAIN_X)

616800
616800
616800
616800


Load the Validation files. Do not run this if you want to calculate SNR.

DATA_val_s = load_file(VAL_S)
DATA_val_n = load_file(VAL_N)
DATA_val_x_cmplx = load_file(VAL_X_cmplx)
DATA_val_x = load_file(VAL_X)

Load the Test files. Do not run this if you want to creat wav output files.

DATA_test_x_cmplx = load_file(TEST_X_cmplx)
DATA_test_x = load_file(TEST_X)

### Training and Validation Labels (IRM)

In [12]:
DATA_train_M=[ (1.0*(DATA_train_s[i]>DATA_train_n[i])) for i in range(len(DATA_train_s)) ]

DATA_val_M=[ (1.0*(DATA_val_s[i]>DATA_val_n[i])) for i in range(len(DATA_val_s)) ]
    

### Batch Generator with Mini-Batch

In [13]:
def next_batch_mb(X_,Y_,mini_batch):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)
        
        if e>0 and (e+1)%10==0:

            batch_x, temp_x = None, batch_x
            batch_y, temp_y = None, batch_y
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))
            
            total_mini_batch = temp_x.shape[0]//mini_batch
     
            for mb in range(total_mini_batch):
                start_b = (mb*mini_batch)
                end_b = ((mb+1)*mini_batch)

                yield temp_x[start_b:end_b],temp_y[start_b:end_b]

    

### Batch Generator

In [14]:
def next_batch(X_,Y_):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)

        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_y, batch_y = batch_y, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))
        
            yield temp_x,temp_y

### Model 1: Single Layer RNN (GRU)

### Model 2: Single Layer RNN (GRU) with Mini-batch (10)

### Model 3: Two layer Bi-directional RNN (GRU), droupout=0.2 

In [17]:
model = Sequential()

model.add(Bidirectional(GRU(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(Max_RNN, return_sequences=True)))

model.add(Dense(513, activation='sigmoid'))

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

for e in range(5):
    for (b_x,b_y), (v_x,v_y) in zip(next_batch(DATA_train_x, DATA_train_M), next_batch(DATA_val_x, DATA_val_M)):
        model.fit(b_x, b_y, validation_data=(v_x,v_y), shuffle=True, batch_size=256)

        
# Final evaluation of the model
scores = []
for v_x,v_y in next_batch(DATA_val_x, DATA_val_x):
    scores.append( model.evaluate(v_x, v_y,verbose=0)[1] )

print("Accuracy: %.2f%%" % np.mean(scores))

Accuracy: 0.01%


### Batch Generator for SNR 

In [23]:
def next_batch_SXX_cmplxS_cmplx(S_,X_,X_cmplx_,S_cmplx_):
    
    batch_s = None
    batch_x = None
    batch_x_cmplx = None
    batch_s_cmplx = None
    
    for e,(s,x,x_cmplx,s_cmplx) in enumerate( zip(S_,X_,X_cmplx_,S_cmplx_)): 
        batch_s = np.array(s.T) if batch_s is None else np.concatenate( (batch_s,s.T), axis=0)
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_x_cmplx = np.array(x_cmplx.T) if batch_x_cmplx is None else np.concatenate( (batch_x_cmplx,x_cmplx.T), axis=0)
        batch_s_cmplx = np.array(s_cmplx.T) if batch_s_cmplx is None else np.concatenate( (batch_s_cmplx,s_cmplx.T), axis=0)
        
        
        if e>0 and (e+1)%10==0:
            temp_s, batch_s = batch_s, None
            temp_x, batch_x = batch_x, None
            temp_x_cmplx, batch_x_cmplx = batch_x_cmplx, None
            temp_s_cmplx, batch_s_cmplx = batch_s_cmplx, None
            
            temp_s = temp_s.reshape((-1,Max_RNN,513))
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_x_cmplx = temp_x_cmplx.reshape((-1,Max_RNN,513))

            yield temp_s,temp_x,temp_x_cmplx,temp_s_cmplx

### Calculate SNR

In [ ]:
sum_s = 0.0
sum_s_diff = 0.0


for v_s,v_x,v_x_cmplx,v_s_cmplx in next_batch_SXX_cmplxS_cmplx(DATA_val_s,DATA_val_x,DATA_val_x_cmplx,DATA_val_s_cmplx):
    
    mask = model.predict(v_x)
    S_hat = (mask) * v_x_cmplx
    S_hat = S_hat.reshape(-1,513).T
    S = v_s_cmplx.T

    S_org = librosa.istft(S, hop_length=512)
    S_pred = librosa.istft(S_hat, hop_length=512)

    sum_s += np.sum(S_org*S_org)
    sum_s_diff += np.sum((S_org-S_pred)*(S_org-S_pred))
    
acc = sum_s/ sum_s_diff
print( 'SNR:', 10*np.log10(acc) )

### Write Audio files

In [37]:
def write_audio():
    mags = None
    cmplxs = None
    
    for e, file_x in enumerate(fname_test[:10]):
        print(e)
        sn, sr = librosa.load(file_x, sr=None)
        Sn = librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
        
        mags = np.array(mag_Sn.T) if mags is None else np.concatenate( (mags,mag_Sn.T), axis=0)
        cmplxs = np.array(Sn.T) if cmplxs is None else np.concatenate( (cmplxs,Sn.T), axis=0)
        
        if e>0 and (e+1)%10==0:
            temp, mags = mags, None
            
            temp = temp.reshape((-1,Max_RNN,513))
            mask = model.predict(temp)
            
            mask=mask.reshape(-1,513)
            S_hat = (mask) * cmplxs
            S_hat = S_hat.T
            
            lenght_w = S_hat.shape[1]//10
            for clip in range(10):
                start_w = clip*lenght_w
                end_w = (clip+1)*lenght_w
                
                wav = S_hat[:,start_w:end_w].T
                S_time=librosa.istft(wav, hop_length=512)
                fname = PATH_directory+PATH_denoise+ e + "_redoise.wav"
                
                print(audio_fname)
                librosa.output.write_wav(fname, S_time, sr)
                

In [38]:
write_audio()

In [10]:
np.save('DATA_val_sX.npy', DATA_val_sX)
np.save('DATA_val_xX.npy', DATA_val_xX)

In [50]:
DATA_val_sX_f = np.load('DATA_val_sX.npy')
DATA_val_xX_f = np.load('DATA_val_xX.npy')

DATA_test_x (513, 44150)
DATA_test_x (513, 5, 8830)


### Save Dataset in files

In [3]:
np.save('DATA_train_s.npy', DATA_train_s)
np.save('DATA_train_n.npy', DATA_train_n)
np.save('DATA_train_x.npy', DATA_train_x)
np.save('DATA_train_M.npy', DATA_train_M)


np.save('DATA_val_s.npy', DATA_val_s)
np.save('DATA_val_n.npy', DATA_val_n)
np.save('DATA_val_x.npy', DATA_val_x)
np.save('DATA_val_M.npy', DATA_val_M)

np.save('DATA_test_x.npy', DATA_test_x)

np.save('DATA_val_sX.npy', DATA_val_sX)
np.save('DATA_val_xX.npy', DATA_val_xX)

NameError: name 'DATA_train_s' is not defined

### Load Dataset in files

In [3]:
DATA_train_s_f = np.load('DATA_train_s.npy')
DATA_train_n_f = np.load('DATA_train_n.npy')
DATA_train_x_f = np.load('DATA_train_x.npy')
DATA_train_M_f = np.load('DATA_train_M.npy')

DATA_val_s_f = np.load('DATA_val_s.npy')
DATA_val_n_f = np.load('DATA_val_n.npy')
DATA_val_x_f = np.load('DATA_val_x.npy')
DATA_val_M_f = np.load('DATA_val_M.npy')

DATA_test_x_f = np.load('DATA_test_x.npy')

DATA_val_sX_f = np.load('DATA_val_sX.npy')
DATA_val_xX_f = np.load('DATA_val_xX.npy')

In [4]:
print('DATA_train_s_f', DATA_train_s_f.shape)
print('DATA_train_M_f', DATA_train_M_f.shape)
print('DATA_val_x_f', DATA_val_x_f.shape)
print('DATA_test_x_f', DATA_test_x_f.shape)
print('DATA_val_xX_f', DATA_val_xX_f.shape)

DATA_train_s_f (513, 5, 23710)
DATA_train_M_f (513, 5, 23710)
DATA_val_x_f (513, 5, 23710)
DATA_test_x_f (513, 5, 8830)
DATA_val_xX_f (513, 5, 23710)


In [5]:
DATA_train_s_f = DATA_train_s_f.reshape(513,-1)
print('DATA_train_s_f', DATA_train_s_f.shape)

DATA_train_s_f = DATA_train_s_f.T
print('DATA_train_s_f', DATA_train_s_f.shape)

DATA_train_s_f = DATA_train_s_f.reshape(-1,5,513)
print('DATA_train_s_f', DATA_train_s_f.shape)

DATA_train_s_f (513, 118550)
DATA_train_s_f (118550, 513)
DATA_train_s_f (23710, 5, 513)


In [6]:
DATA_train_n_f = (DATA_train_n_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_train_x_f = (DATA_train_x_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_train_M_f = (DATA_train_M_f.reshape(513,-1).T).reshape(-1,5,513)

DATA_val_s_f = (DATA_val_s_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_n_f = (DATA_val_n_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_x_f = (DATA_val_x_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_M_f = (DATA_val_M_f.reshape(513,-1).T).reshape(-1,5,513)

DATA_test_x_f = (DATA_test_x_f.reshape(513,-1).T).reshape(-1,5,513)

DATA_val_sX_f = (DATA_val_sX_f.reshape(513,-1).T).reshape(-1,5,513)
DATA_val_xX_f = (DATA_val_xX_f.reshape(513,-1).T).reshape(-1,5,513)

In [7]:
print('DATA_train_s_f', DATA_train_s_f.shape)
print('DATA_train_M_f', DATA_train_M_f.shape)
print('DATA_val_x_f', DATA_val_x_f.shape)
print('DATA_test_x_f', DATA_test_x_f.shape)
print('DATA_val_xX_f', DATA_val_xX_f.shape)

DATA_train_s_f (23710, 5, 513)
DATA_train_M_f (23710, 5, 513)
DATA_val_x_f (23710, 5, 513)
DATA_test_x_f (8830, 5, 513)
DATA_val_xX_f (23710, 5, 513)


In [22]:
tr_categorical_labels = to_categorical(DATA_train_M_f, num_classes=2)
print(tr_categorical_labels.shape)

v_categorical_labels = to_categorical(DATA_val_M_f, num_classes=2)
print(v_categorical_labels.shape)

(23710, 5, 513, 2)
(23710, 5, 513, 2)


### RNN Model

In [8]:
Max_RNN = 5

In [10]:
# create the model

model = Sequential()

model.add( Bidirectional(GRU(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513) ))
model.add(Dropout(0.1))
model.add(Dense(513, activation='sigmoid'))
          
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# model.fit(DATA_train_x_f.T, tr_categorical_labels, validation_data=(DATA_val_x_f.T,v_categorical_labels), shuffle=True, nb_epoch=100, batch_size=10)
# # Final evaluation of the model
# scores = model.evaluate(DATA_val_x_f.T, v_categorical_labels, verbose=0)



model.fit(DATA_train_x_f, DATA_train_M_f, validation_data=(DATA_val_x_f,DATA_val_M_f), shuffle=True, nb_epoch=10, batch_size=10)

# model.fit(DATA_train_x_f, tr_categorical_labels, validation_data=(DATA_val_x_f,v_categorical_labels), shuffle=True, nb_epoch=10, batch_size=10)
# Final evaluation of the model
scores = model.evaluate(DATA_val_x_f, DATA_val_M_f, verbose=0)



print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 5, 10)             15570     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5, 513)            5643      
Total params: 21,213
Trainable params: 21,213
Non-trainable params: 0
_________________________________________________________________
None


/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 23710 samples, validate on 23710 samples
Epoch 1/10
23710/23710 [==============================] - 55s 2ms/step - loss: 0.5898 - acc: 0.6839 - val_loss: 0.5429 - val_acc: 0.7291
Epoch 2/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5468 - acc: 0.7229 - val_loss: 0.5234 - val_acc: 0.7439
Epoch 3/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5346 - acc: 0.7325 - val_loss: 0.5102 - val_acc: 0.7529
Epoch 4/10
23710/23710 [==============================] - 53s 2ms/step - loss: 0.5267 - acc: 0.7385 - val_loss: 0.5061 - val_acc: 0.7552
Epoch 5/10
23710/23710 [==============================] - 53s 2ms/step - loss: 0.5212 - acc: 0.7429 - val_loss: 0.5035 - val_acc: 0.7576
Epoch 6/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5175 - acc: 0.7459 - val_loss: 0.4956 - val_acc: 0.7625
Epoch 7/10
23710/23710 [==============================] - 52s 2ms/step - loss: 0.5145 - acc: 0.7477 - val_loss: 0.4921 - val_acc

In [ ]:
model = Sequential()

model.add(GRU(output_dim = 513, input_shape=(Max_RNN,513), return_sequences=True))
# model.add(GRU(output_dim = 513, input_length = 5, input_dim = 513, return_sequences=True))

model.add(Activation('relu'))
model.add(TimeDistributed(Dense(513, activation='sigmoid')))
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

model.fit(DATA_train_x_f, DATA_train_M_f, validation_data=(DATA_val_x_f,DATA_val_M_f), shuffle=True, epochs=10, batch_size=10)

# model.fit(DATA_train_x_f, tr_categorical_labels, validation_data=(DATA_val_x_f,v_categorical_labels), shuffle=True, nb_epoch=10, batch_size=10)
# Final evaluation of the model
scores = model.evaluate(DATA_val_x_f, DATA_val_M_f, verbose=0)



print("Accuracy: %.2f%%" % (scores[1]*100))

/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(input_shape=(5, 513), return_sequences=True, units=513)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 5, 513)            1580553   
_________________________________________________________________
activation_1 (Activation)    (None, 5, 513)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 513)            263682    
Total params: 1,844,235
Trainable params: 1,844,235
Non-trainable params: 0
_________________________________________________________________
None
Train on 23710 samples, validate on 23710 samples
Epoch 1/10


In [8]:
def preprossed_dataX(file_name):
    
    sn, sr=librosa.load(file_name, sr=None)
    X=librosa.stft(sn, n_fft=1024, hop_length=512)
    return X

In [33]:
DATA_val_sX = None
DATA_val_xX = None

for file_s, file_x in zip( sorted(glob.glob(PATH_directory+PATH_val+CLEAN_format_val)), sorted(glob.glob(PATH_directory+PATH_val+MIX_format_val))):
    
    DATA_val_sX = np.array(preprossed_dataX(file_s)) if DATA_val_sX is None else np.concatenate( (DATA_val_sX,preprossed_dataX(file_s)),axis=1)
    DATA_val_xX = np.array(preprossed_dataX(file_x)) if DATA_val_xX is None else np.concatenate( (DATA_val_xX,preprossed_dataX(file_x)),axis=1)
    
    
    
    

In [10]:
print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_sX = DATA_val_sX.reshape( (513,5,-1))
DATA_val_xX = DATA_val_xX.reshape( (513,5,-1))

print('DATA_val_sX',DATA_val_sX.shape)
print('DATA_val_xX',DATA_val_xX.shape)

DATA_val_xX (513, 118550)
DATA_val_xX (513, 5, 23710)


In [ ]:
np.save('DATA_val_sX.npy', DATA_val_sX)
np.save('DATA_val_xX.npy', DATA_val_xX)

In [13]:
DATA_val_sX_f = np.load('DATA_val_sX.npy')
DATA_val_xX_f = np.load('DATA_val_xX.npy')

In [25]:
labels = model.predict(DATA_test_x)
labels.shape

ValueError: Error when checking model : the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 400 arrays: [array([[0.31088, 0.00439, 0.02531, ..., 0.03555, 0.05921, 0.02048],
       [0.20403, 0.21771, 0.24856, ..., 0.07964, 0.19597, 0.11246],
       [0.00054, 0.27443, 0.5295 , ..., 0.14783, 0.20548, 0.119...

In [12]:
S_hat = (labels) * DATA_val_xX_f

In [13]:
# S_org = (DATA_val_s_f).flatten()
# S_pred = S_hat.flatten()
st = DATA_val_sX_f.reshape(513,-1)
st_hat = S_hat.reshape(513,-1)

print(st.shape)
print(st_hat.shape)

S_org = librosa.istft(st, hop_length=512)
S_pred = librosa.istft(st_hat, hop_length=512)

(513, 118550)
(513, 118550)


In [20]:
print(S_pred.shape, S_org.shape)

(60697088,) (60697088,)


In [14]:
acc = np.sum(S_org*S_org)/ np.sum((S_org-S_pred)*(S_org-S_pred))
print(acc)

10*np.log10(acc)

4.1112037


6.139689683914185

In [1]:
# Noisy -> Input Data
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag = np.abs(X)

# Clean -> Label
s, sr=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
S_mag = np.abs(S)

IndentationError: expected an indented block (<ipython-input-1-09b5a3680f5d>, line 9)

### Function for generating next batch

In [56]:
def next_batch(X,Y, batch_size):
    num_samples, _ = X.shape
    
    selected_indics = np.random.randint(num_samples-batch_size)
#     print(selected_indics)
    return X[selected_indics:selected_indics+batch_size], Y[selected_indics:selected_indics+batch_size]

### Global Parameters

In [57]:
NUM_ITERATION = 1000
BATCH_SIZE = X_mag.shape[0]

### Xavier Initialization of Weights
These are the weight initialization function used in defining model.

In [58]:
def weight_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(2.0/sum(shape)) )
    return tf.Variable(initial)

def bias_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(1.0/sum(shape)) )
    return tf.Variable(initial)

### Create the fully connected model 

In [59]:
x = tf.placeholder(tf.float32, [None, 513]) 

W_1 = weight_variable([513, 1024])
b_1 = bias_variable([1024])

W_2 = weight_variable([1024, 1024])
b_2 = bias_variable([1024])

W_3 = weight_variable([1024, 1024])
b_3 = bias_variable([1024])

W_4 = weight_variable([1024, 1024])
b_4 = bias_variable([1024])

W_5 = weight_variable([1024, 513])
b_5 = bias_variable([513])

y_ = tf.placeholder(tf.float32, [None, 513]) # original


# Layer connections and Activation functions
y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)
y_2 = tf.nn.relu(tf.matmul(y_1, W_2) + b_2)
y_3 = tf.nn.relu(tf.matmul(y_2, W_3) + b_3)
y_4 = tf.nn.relu(tf.matmul(y_3, W_4) + b_4)
y =  tf.nn.relu(tf.matmul(y_2, W_5) + b_5) # predicted


# Define loss and optimizer
mse = tf.reduce_sum( tf.losses.mean_squared_error(labels=y_, predictions=y) )
train_step = tf.train.AdamOptimizer().minimize(mse)

### Run the model
We try different batch sizes, but for whole input dimension batch size gives better result and it removes the need of batch normalization. Since the input dimension is not very large, we can do this. For larger input dimensions, we have to adopt mini batch techniques. 

In [60]:
# Configuration to control GPU use
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33
sess = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()


# Train Model
for _ in range(NUM_ITERATION):
    for _ in range(((X_mag.T).shape[0]//BATCH_SIZE)):
        batch_xs, batch_ys = next_batch(X_mag.T,S_mag.T, BATCH_SIZE)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

### Read the Test files

In [61]:
# Load Test data-1
sn, sr=librosa.load('test_x_01.wav', sr=None)
X_test_01=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag_test_01 = np.abs(X_test_01)

# Load Test data-2
sn, sr=librosa.load('test_x_02.wav', sr=None)
X_test_02=librosa.stft(sn, n_fft=1024, hop_length=512)
X_mag_test_02 = np.abs(X_test_02)

print(X_mag_test_01.shape)

(513, 142)


### Save the Test Output Files

In [62]:
# Test model-1
S_hat_mag_test_01=sess.run(y, feed_dict={x: X_mag_test_01.T})
S_hat_test_01=(X_test_01/X_mag_test_01)*S_hat_mag_test_01.T
S_hat_01=librosa.istft(S_hat_test_01, hop_length=512)
librosa.output.write_wav('test_s_01_recons.wav', S_hat_01, sr)

# Test model-2
S_hat_mag_test_02=sess.run(y, feed_dict={x: X_mag_test_02.T})
S_hat_test_02=(X_test_02/X_mag_test_02)*S_hat_mag_test_02.T
S_hat_02=librosa.istft(S_hat_test_02, hop_length=512)
librosa.output.write_wav('test_s_02_recons.wav', S_hat_02, sr)